In [241]:
import requests
import json
import pprint
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import csv
import re
from datetime import datetime
import time
import numpy as np

# I. Lấy thông tin chung
- Lấy thông tin của tất cả các quán ăn thuộc 1 tỉnh/TP. Rộng hơn là thông tin tất cả quán ăn của nhiều tỉnh thành
- Với mỗi quán ăn, lấy những trường thông tin cơ bản như
    + name, addr, CityId, RestaurantId, RestaurantStatus <font color=green>// thông tin cơ bản</font>
    + AvgRating
    + TotalPictures, TotalSaves, TotalReviews, TotalFavourites, TotalViews, TotalCheckIns
    + CreatedOn <font color=green>//ngày shop dc tạo đầu tiên trên foody (trường này bị sao rồi, toàn mặc định)</font>
    + Latitude & Longitude <font color=green>//toạ độ</font>
    + ResUrlReviews <font color=green>//URL dẫn tới trang chứa bình luận của shop đó</font>


In [3]:
def get_single_page(page, st, dist):
    '''extract 1 page'''
    url = f'https://www.foody.vn/ho-chi-minh/dia-diem?vt=row&st={st}&dt={dist}&page={page}'
    all_shops_API = requests.get(url, headers={'X-Requested-With': 'XMLHttpRequest',
                                               'Cookie': 'FOODY.AUTH=A3F33DD0BA0DC443960AC294DA1D898EE203A64E31B64523D3674F29D60EF3C3C900F1F3BF70DAC47FD0ED268F012BD8FD06CCA143E3FCC2F79AA22CFF031D91A06F5D97B96B09529CF8E5A06E58BCF34312AFDA1F37E6B39CC56A6BA525D72DEDDF1EFE136334F777A1AD191BC3B393CA5D7B3008AA54A9565C21793E196AB15D19E8867327553F696DDCA2938C22208B84620BAA2F757C9D4B8E2FF3CE8A28E6EBC891AAE2F227C02CF6834427BD815163F0DAB98F2E47CECEA44EE3CB88F9A94D33123EEA15B660CF6DE09385DF9EAFD8741C8B4936BF4B5C40B003150E622D303BAC626EBF7FD60BCB19647A46FC'})
    data = all_shops_API.text

    # handle errors
    try:
        json_data = json.loads(data)
        search_items = json_data['searchItems']
    except: # no data
        return []
    
    # get basic shop's info
    shop_attribs = []
    
    for shop in search_items: 
        shop_attribs.append({'ReviewUrl' : shop['ReviewUrl'],
                             'name': shop['Name'],
                             'Address': shop['Address'],
                             'districtid': shop['DistrictId'],
                             'RestaurantId': shop['Id'],
                             'RestaurantStatus': shop['Status'],
                             'avgrating' : shop['AvgRatingOriginal'],
                             'TotalPictures' : shop['TotalPictures'],
                             'TotalSaves' : shop['TotalSaves'],
                             'TotalReviews' : shop['TotalReview'],
                             'TotalFavourites' : shop['TotalFavourite'],
                             'TotalViews' : shop['TotalView'],
                             'TotalCheckIns' : shop['TotalCheckins'],
                             'Latitude'  : shop['Latitude'],
                             'Longitude'  : shop['Longitude']})
    
    return shop_attribs # [{},{},..]

def get_single_district(dist=1):
    '''get all shops in a single district'''
    page = 1
    all_shop = []
    empty = False

    for page in range(1, 101):
        if empty:
            break
        for st in range(1, 60):
            time.sleep(1)
            one_page = get_single_page(page, st, dist) 
            
            if not one_page: # check empty
                empty = True
                break
            all_shop.extend(one_page)
    return all_shop # [{},{},..]                       

def save_district_to_file(dist):
    shops_in_a_district = get_single_district(dist)
    df = pd.DataFrame.from_dict(shops_in_a_district) 
    df.dropna(subset='ReviewUrl').drop_duplicates().to_csv(f'q_id_{dist}.csv', index = False, header=True, encoding='utf-8')

In [4]:
all_dist = '1,4,5,6,7,8,9,10,11,12,13,14,15,16,17,19,2,18,693,696,699,695,694,698'.split(',')
for dist in all_dist:
    save_district_to_file(dist)
    print(f'save district_id = {dist} successfully')

save district_id = 12 successfully
save district_id = 13 successfully
save district_id = 14 successfully


## Merge data của tất cả các quận

- Merge các file quận/huyện độc lập thành 1 dataframe duy nhất `df_all` 

In [85]:
import os

df_all = pd.DataFrame()
col_order = list(pd.read_csv('crawled_all_shops/q1.csv'))
for file in os.listdir('crawled_all_shops/'):
    df_dist = pd.read_csv(f"crawled_all_shops/{file}")
    df_dist.reindex(col_order, axis=1) # df_dist's col order must be unified
    df_all = pd.concat([df_all, df_dist])

In [122]:
print(df_all.shape)
print("drop_dup(ReviewUrl) + drop_na(ReviewUrl):", df_all.drop_duplicates('ReviewUrl').dropna(subset='ReviewUrl').shape)

(58365, 15)
drop_dup(ReviewUrl) + drop_na(ReviewUrl): (58365, 15)


In [104]:
# rename columns
df_all = df_all.drop_duplicates('ReviewUrl').dropna(subset='ReviewUrl')
df_all.rename(columns={'name': 'Name', 'avgrating': 'AvgRating', 'districtid': 'District'}, inplace=True)
df_all.to_csv(f'crawled_all_shops/all_dist.csv', index = False, header=True, encoding='utf-8')

In [ ]:
# đổi df['District'] từ mã encode quận sang quận bằng chữ
all_dist_encode = '1,4,5,6,7,8,9,10,11,12,13,14,15,16,17,19,2,18,693,696,699,695,694,698'.split(',')
all_dist = '1,2,3,4,5,6,7,8,9,10,11,12,Bình Thạnh,Tân Bình,Phú Nhuận,Tân Phú,Gò Vấp,Bình Tân,Tp. Thủ Đức,Huyện Bình Chánh,Huyện Nhà Bè,Huyện Hóc Môn,Huyện Củ Chi,Huyện Cần Giờ'.split(',')
df_all['District'] = df_all['District'].apply(lambda x: all_dist[all_dist_encode.index(str(int(x)))] if x == x else "")
df_all.head()

- Merge chung với file Data_TPHCM đang chứa 69k+ dòng

In [203]:
# đọc Data_TPHCM & bsung các gtrị rỗng vào các empty_cols
data_hcm = pd.read_csv('crawled_all_shops/Data_TPHCM.csv')[:-1] # drop 'city' col
empty_cols = list(set(list(df_all)).difference(set(list(data_hcm))))
data_hcm[empty_cols] = np.nan 
data_hcm = data_hcm.reindex(list(df_all), axis=1)
# data_hcm.head()

In [204]:
# preprocess 
data_hcm['District'] = data_hcm['District'].apply(lambda x: x[2:-2])

In [205]:
# merge data
data_hcm = pd.concat([data_hcm, df_all])
data_hcm = data_hcm.drop_duplicates('ReviewUrl').dropna(subset='ReviewUrl')
data_hcm.shape

(93669, 15)

In [206]:
# save merged data to csv
data_hcm.to_csv(f'crawled_all_shops/data_hcm.csv', index = False, header=True, encoding='utf-8')

# II. Lấy thông tin cụ thể từng quán ăn
Thông tin lần này chia thành 2 loại:
- Thông tin thực đơn (Get delivery dishes)
    - dish_type_name <font color=green>//Loại đồ ăn: 1 loại đồ ăn gồm nhiều món ăn, VD: bún thì có bún cá, bún chả</font>
    - total_like <font color=green>//tổng lượt like loại đồ ăn đó</font>
    - is_available <font color=green>//loại đồ ăn đó còn món ko</font>
    - dish_name <font color=green>//món ăn thuộc loại đồ ăn đó. VD: bún cá thập cẩm</font>
    - dish_price <font color=green>//giá 1 món ăn thuộc 1 loại đồ ăn </font>
    - options: <font color=green>// có option nào cho món ăn đó. VD: có option chọn thêm topping như rau, cua,... </font>
        [
        option_1: name, price;
        option_2: name, price;
        ....
        ]
    - option_min_select, option_max_select

- Thông tin chi tiết quán ăn (Get detail):
    - phone 
	- district_id
	- category <font color=green>//loại hình quán ăn?</font>
	- cuisines <font color=green>//quán ăn thuộc phân khúc món nào? món Việt, Âu,...?</font>
	- service_fee <font color=green>//fí dịch vụ</font>
	- avg_price <font color=green>// chắc là giá trung bình của quán ăn.</font>
	- min_order_value <font color=green>//fải mua tối thiểu bnhiêu tiền?</font>
	- promotion <font color=green>// trong này nhiều trường nhỏ, tự nghiên cứu thêm</font>
	- min_charge <font color=green>//fí fụ thu nhỏ nhất</font>
	- week_days, operating <font color=green>//thời gian mở cửa</font>
	- min_shipping_fee <font color=green>// có thể là giá ship thấp nhất</font>
	- is_foody_delivery <font color=green>// có thể là shop này có phải là shop bán mang về hay không</font>
	- price_range <font color=green>// khoảng giá min – max của quán.</font>

---

- Lấy **menu** của 1 quán ăn: xây dựng hàm nhận vào `id_quán_ăn` & get về thông tin menu

In [ ]:
# globals
menu_df = pd.DataFrame(columns=['dish_type_id', 'dish_type_name'])
menu_dish_df = pd.DataFrame(columns=['dish_type_id', 'dish_id', 'dish_name', 'dish_description', 'dish_price_value', 'dish_total_like', 'dish_is_available'])
menu_dish_option_df = pd.DataFrame(columns=['dish_type_id', 'dish_id', 'option_id', 'option_min_select', 'option_max_select'])
menu_dish_option_item_df = pd.DataFrame(columns=['dish_type_id', 'dish_id', 'option_id', 'item_id', 'item_name', 'item_price'])

def get_delivery_dish(restaurant_id):
	global menu_df, menu_dish_df, menu_dish_option_df, menu_dish_option_item_df
	delivery_dish_API = requests.get(f"https://gappapi.deliverynow.vn/api/dish/get_delivery_dishes?request_id={restaurant_id}&id_type=1", 
									 headers={'x-foody-api-version': '1',
											'x-foody-app-type': '1004',
											'x-foody-client-language': 'vi',
											'x-foody-client-type': '1',
											'x-foody-client-version': '1',
											'X-Foody-Client-Id': 'cookies.__ondemand_sessionid'
											}
									)
	data = delivery_dish_API.text
	try:
		dataJson = json.loads(data)
		menu_infos = dataJson['reply']['menu_infos']
	except:
		return

	menu_df_temp = pd.DataFrame(columns=['restaurant_id', 'dish_type_id', 'dish_type_name'])
	menu_dish_df_temp = pd.DataFrame(columns=['restaurant_id', 'dish_type_id', 'dish_id', 'dish_name', 'dish_description', 'dish_price_value', 'dish_total_like', 'dish_is_available'])
	menu_dish_option_df_temp = pd.DataFrame(columns=['restaurant_id', 'dish_type_id', 'dish_id', 'option_id', 'option_min_select', 'option_max_select'])
	menu_dish_option_item_df_temp = pd.DataFrame(columns=['restaurant_id', 'dish_type_id', 'dish_id', 'option_id', 'item_id', 'item_name', 'item_price'])

	for menu_info in menu_infos:
		menu_df_temp.loc[len(menu_df_temp.index)] = [restaurant_id, menu_info['dish_type_id'], menu_info['dish_type_name']]
		for dish in menu_info['dishes']:
			menu_dish_df_temp.loc[len(menu_dish_df_temp.index)] = [
					restaurant_id,
					menu_info['dish_type_id'],
					dish['id'],
					dish['name'],
					dish['description'],
					dish['price']['value'],
					dish['total_like'],
					dish['is_available']
			]
			for option in dish['options']:
				menu_dish_option_df_temp.loc[len(menu_dish_option_df_temp.index)] = [
						restaurant_id,
						menu_info['dish_type_id'],
						dish['id'],
						option['id'],
						option['option_items']['min_select'],
						option['option_items']['max_select']
				]
				for item in option['option_items']['items']:
					menu_dish_option_item_df_temp.loc[len(menu_dish_option_item_df_temp.index)] = [
							restaurant_id,
							menu_info['dish_type_id'],
							dish['id'],
							option['id'],
							item['id'],
							item['name'],
							item['price']['value']
					]
								
	menu_df = pd.concat([menu_df, menu_df_temp])
	menu_dish_df = pd.concat([menu_dish_df, menu_dish_df_temp])
	menu_dish_option_df = pd.concat([menu_dish_option_df, menu_dish_option_df_temp])
	menu_dish_option_item_df = pd.concat([menu_dish_option_item_df, menu_dish_option_item_df_temp])
# TEST
data_hcm = pd.read_csv('crawled_all_shops/data_hcm.csv')
data_hcm['RestaurantId'] = data_hcm['RestaurantId'].apply(lambda x: str(x).split('.')[0] if x == x else np.nan)
cnt = 0

for i, res_id in enumerate(data_hcm['RestaurantId']):
	if res_id == res_id: # check nan
		get_delivery_dish(res_id)

In [ ]:
# export to csv
menu_df.to_csv('crawled_menu/menu.csv', sep=',', encoding='utf-8', index = False)
menu_dish_df.to_csv('crawled_menu/menu.dish.csv', sep=',', encoding='utf-8', index = False)
menu_dish_option_df.to_csv('crawled_menu/menu.dish.option.csv', sep=',', encoding='utf-8', index = False)
menu_dish_option_item_df.to_csv('crawled_menu/menu.dish.option.item.csv', sep=',', encoding='utf-8', index = False)

- Lấy **detail** của 1 quán ăn: xây hàm nhận vào `id_quán_ăn` và lưu thông tin chi tiết quán đó (VD: min_charge, rating_avg, min_shipping_fee,...) vào file csv

In [ ]:
detail_df = pd.DataFrame(columns=['city_id', 'district_id', 'request_id', 'categories', 'cuisines', 'total_review', 'avg', \
                'service_fee', 'avg_price', 'min_order_value', 'min_charge', 'minimun_shiping_fee', 'is_foody_delivery', 'min_price', 'max_price'])

def get_detail(restaurant_id):
    detail_API = requests.get(f"https://gappapi.deliverynow.vn/api/delivery/get_detail?request_id={restaurant_id}&id_type=1", 
                                headers={'x-foody-api-version': '1',
                                        'x-foody-app-type': '1004',
                                        'x-foody-client-language': 'vi',
                                        'x-foody-client-type': '1',
                                        'x-foody-client-version': '1',
                                        'X-Foody-Client-Id': 'cookies.__ondemand_sessionid'
                                        }
                            )
    data = detail_API.text
    dataJson = json.loads(data)
    if not dataJson['reply']:
        return
    detail_infos = dataJson['reply']['delivery_detail']

    detail_df_temp = pd.DataFrame(columns=['city_id', 'district_id', 'request_id', 'categories', 'cuisines', 'total_review', 'avg', \
                'service_fee', 'avg_price', 'min_order_value', 'min_charge', 'minimun_shiping_fee', 'is_foody_delivery', 'min_price', 'max_price'])
    detail_rating_df_temp = pd.DataFrame(columns=['request_id', 'total_review', 'avg'])
    detail_delivery_df_temp = pd.DataFrame(columns=['request_id','service_fee', 'avg_price', 'min_order_value', 'min_charge', 'minimun-shiping-fee', 'is_foody_delivery'])
    detail_price_df_temp = pd.DataFrame(columns=['request_id', 'min', 'max'])
    
    detail_df_temp.loc[len(detail_df_temp.index)] = [
        detail_infos['city_id'], detail_infos['district_id'], restaurant_id, detail_infos['categories'], \
        detail_infos['cuisines'], detail_infos['rating']['total_review'], detail_infos['rating']['avg'], detail_infos['delivery']['service_fee']['value'],\
        detail_infos['delivery']['avg_price']['value'], detail_infos['delivery']['min_order_value']['value'], detail_infos['delivery']['min_charge'], \
        detail_infos['delivery']['shipping_fee']['minimum_fee'], detail_infos['delivery']['is_foody_delivery'], \
        detail_infos['price_range']['min_price'], detail_infos['price_range']['max_price']
    ]

    detail_df = pd.concat([detail_df, detail_df_temp])

for i, res_id in enumerate(data_hcm['RestaurantId']):
    if cnt > 100:
        break
    if res_id == res_id: # check nan
        cnt += 1
        get_detail(res_id)

In [ ]:
shop_df = pd.DataFrame(columns=['shop_id', 'name', 'short_description', 'city_id', 'district_id', 'address', 
                                'rating_total_review', 'rating_avg', 'rating_app_link',  
                                'service_fee', 'avg_price', 'min_order_value', 'min_charge', 'min_shipping_fee', 'min_price', 'max_price', 
                                'is_foody_delivery', 'week_days', 'start_time', 'end_time'])
shop_phone_df = pd.DataFrame(columns=['shop_id', 'phone'])
shop_category_df = pd.DataFrame(columns=['shop_id', 'category'])
shop_cuisine_df = pd.DataFrame(columns=['shop_id', 'cuisine'])
shop_promotion_df = pd.DataFrame(columns=['shop_id', 'promo_code', 'discount_amount', 'min_order_amount', 'max_discount_amount', 'apply_order','max_usage_time', 'expired'])


def get_detail(restaurant_id):
        delivery_dish_API = requests.get(f"https://gappapi.deliverynow.vn/api/delivery/get_detail?request_id={restaurant_id}&id_type=1", 
                                        headers={'x-foody-api-version': '1',
                                                'x-foody-app-type': '1004',
                                                'x-foody-client-language': 'vi',
                                                'x-foody-client-type': '1',
                                                'x-foody-client-version': '1',
                                                'X-Foody-Client-Id': 'cookies.__ondemand_sessionid'
                                                }
                                        )
        data = delivery_dish_API.text
        dataJson = json.loads(data)
        shop = dataJson['reply']['delivery_detail']

        shop_df.loc[len(shop_df.index)] = [
                shop['id'], shop['name'], shop['short_description'], shop['city_id'], shop['district_id'], shop['address'],
                shop['rating']['total_review'], shop['rating']['avg'], shop['rating']['app_link'],
                shop['delivery']['service_fee']['value'], shop['delivery']['avg_price']['value'], shop['delivery']['min_order_value']['value'],
                shop['delivery']['min_charge'], shop['delivery']['shipping_fee']['minimum_fee'],shop['price_range']['min_price'],
                shop['price_range']['max_price'],shop['delivery']['is_foody_delivery'],
                len(shop['delivery']['time']['week_days']), 
                shop['delivery']['time']['week_days'][0]['start_time'], 
                shop['delivery']['time']['week_days'][0]['end_time'] 
        ]
        
        for phone in shop['phones']:
                shop_phone_df.loc[len(shop_phone_df.index)] = [
                        shop['id'],
                        phone
                ]

        for category in shop['categories']:
                shop_category_df.loc[len(shop_category_df.index)] = [
                        shop['id'],
                        category
                ]                     

        for cuisine in shop['cuisines']:
                shop_cuisine_df.loc[len(shop_cuisine_df.index)] = [
                        shop['id'],
                        cuisine
                ]    

        for promotion in shop['delivery']['promotions']:
                shop_promotion_df.loc[len(shop_promotion_df.index)] = [
                        shop['id'], promotion['promo_code'], promotion['discount_amount'], promotion['min_order_amount'], 
                        promotion['max_discount_amount'], promotion['apply_order'], promotion['user_condition']['limit_per_user']['max_usage_time'], promotion['expired']
                ]


for i in qbt_df['RestaurantId']:
    get_detail(i)



shop_df.to_csv('shop.csv', sep=',', encoding='utf-8')
shop_phone_df.to_csv('shop.phone.csv', sep=',', encoding='utf-8')
shop_category_df.to_csv('shop.category.csv', sep=',', encoding='utf-8')
shop_cuisine_df.to_csv('shop.cuisine.csv', sep=',', encoding='utf-8')
shop_promotion_df.to_csv('shop.promotion.csv', sep=',', encoding='utf-8')